In [29]:
import re 
import nltk 

In [3]:
## Reading data from the text files 

train_data = open("./Data/train.txt").readlines()
val_data = open("./Data/val.txt").readlines()
test_data = open("./Data/test.txt").readlines()


In [7]:
print(len(train_data))
print(len(val_data))
print(len(test_data))


16000
2000
2000


In [8]:
complet_data = train_data + test_data + val_data 
print(len(complet_data))

20000


In [14]:
x=[]
y=[]
for temp in complet_data:
    complete = temp.split(';')
    if len(complete)==2:
        x.append(complete[0])
        y.append(complete[1].strip())

In [21]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
labels 

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']

In [ ]:
## Text cleaning 
# 1. lower case convert all message 
# 2. a-z0-9   other characters 
# 3. 

In [ ]:
def text_cleaning(sentences):
    for sentece in sentences: 
        message = sentece.lower()
        message = re.sub('[^a-z0-9 ]',"",message)
        ls_of_words = nltk.word_tokenize(message)
        
        




cleaned_data = text_cleaning(x)


In [31]:
single_message = x[1].lower()
single_message = re.sub('[^a-z0-9 ]',"",single_message)
ls_of_words = nltk.word_tokenize(single_message)  # single_message.split()